In [ ]:
from pycocotools.coco import COCO # COCO python library
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

import random
import string

import cv2
import os
from pickle import dump, load
import json

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input, Dropout, Attention
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

#from keras.layers.merge import add
from keras.layers import Concatenate
from tensorflow.keras.models import Model, load_model

# small library for seeing the progress of loops.
from tqdm.notebook import tqdm

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from transformers import AutoProcessor,BlipForConditionalGeneration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
coco=COCO("/content/drive/MyDrive/msdata/instances_train2017.json")
annFile = "/content/drive/MyDrive/annotations/captions_train2017.json"
coco_anns=COCO(annFile)#laoding the annotations file

loading annotations into memory...
Done (t=30.39s)
creating index...
index created!
loading annotations into memory...
Done (t=2.86s)
creating index...
index created!


In [ ]:
from torchvision.datasets import CocoCaptions

In [ ]:
from nltk.translate.bleu_score import corpus_bleu


In [ ]:
from torch.utils.data import DataLoader

In [ ]:
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
rouge_scores = []

In [ ]:
import torch

In [ ]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=1fe889348216f76d877d88e621105c332a5b46b7756c99fb26522eb82b742638
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import rouge_score

ModuleNotFoundError: No module named 'rouge_score'

In [ ]:
cats = coco.loadCats(coco.getCatIds())# loading the main cats


maincategories = list(set([cat['supercategory'] for cat in cats]))

In [ ]:
subcategories = [cat['name'] for cat in cats]

print("Number of sub categories: ", len(subcategories))
print("List of sub categories: ", subcategories)

Number of sub categories:  80
List of sub categories:  ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [ ]:
catIds = coco.getCatIds(catNms=subcategories) #getting ids from sub

subcategories_Ids = dict()
for i in range(0,len(subcategories)):
    subcategories_Ids[subcategories[i]] = catIds[i] #getting ids and making the dict then assiging the cat ids to it

print("Sub categories with IDs :",subcategories_Ids)

Sub categories with IDs : {'person': 1, 'bicycle': 2, 'car': 3, 'motorcycle': 4, 'airplane': 5, 'bus': 6, 'train': 7, 'truck': 8, 'boat': 9, 'traffic light': 10, 'fire hydrant': 11, 'stop sign': 13, 'parking meter': 14, 'bench': 15, 'bird': 16, 'cat': 17, 'dog': 18, 'horse': 19, 'sheep': 20, 'cow': 21, 'elephant': 22, 'bear': 23, 'zebra': 24, 'giraffe': 25, 'backpack': 27, 'umbrella': 28, 'handbag': 31, 'tie': 32, 'suitcase': 33, 'frisbee': 34, 'skis': 35, 'snowboard': 36, 'sports ball': 37, 'kite': 38, 'baseball bat': 39, 'baseball glove': 40, 'skateboard': 41, 'surfboard': 42, 'tennis racket': 43, 'bottle': 44, 'wine glass': 46, 'cup': 47, 'fork': 48, 'knife': 49, 'spoon': 50, 'bowl': 51, 'banana': 52, 'apple': 53, 'sandwich': 54, 'orange': 55, 'broccoli': 56, 'carrot': 57, 'hot dog': 58, 'pizza': 59, 'donut': 60, 'cake': 61, 'chair': 62, 'couch': 63, 'potted plant': 64, 'bed': 65, 'dining table': 67, 'toilet': 70, 'tv': 72, 'laptop': 73, 'mouse': 74, 'remote': 75, 'keyboard': 76, 'c

In [ ]:
subcategories_imageIds = dict()

for i in range(0,len(catIds)):
    imgIds = coco.getImgIds(catIds=catIds[i]) #getting the img ids from  then calculating the imgs ids for each cat

    img = []
    for j in imgIds:
        img.append(j)

    subcategories_imageIds[subcategories[i]] = img #getting the image total ids for each sub cat then
    #assign the imgs ids to sub catIMGids=img cat belong to the main cat


In [ ]:
train_cats = subcategories_imageIds['bench'] + subcategories_imageIds['apple'] + subcategories_imageIds['banana']
imgIdss = coco.getImgIds(imgIds = train_cats)
next_pix = imgIdss

In [ ]:
from PIL import Image

In [ ]:
list_of_caps=[]
ann_caps=[]

In [ ]:
for x in range(0,100):
    # Load the image from the COCO dataset
    img = coco.loadImgs(next_pix[x])[0]
    I = io.imread(img['coco_url'])

    # Display the image
    plt.imshow(I)
    plt.axis('off')
    plt.show()

    # Load original captions from the COCO dataset annotations
    annIds = coco_anns.getAnnIds(imgIds=img['id'])
    anns = coco_anns.loadAnns(annIds)
    coco_anns.showAnns(anns)
    ann_caps.append(anns)

    # Generate captions for the image using BLIP
    inputs = processor(I, return_tensors="pt", padding=True)
    out = model.generate(**inputs)

    # Decode the generated caption
    generated_caption = processor.decode(out[0], skip_special_tokens=True)
    list_of_caps.append(generated_caption)
    print("Generated Caption:", generated_caption)



In [ ]:
print(ann_caps[0])

[{'image_id': 294912, 'id': 499175, 'caption': 'The young person is swinging as a ball passes over the plate.'}, {'image_id': 294912, 'id': 500954, 'caption': 'A young person swinging a baseball bat as a ball passes by.'}, {'image_id': 294912, 'id': 504527, 'caption': 'A woman holding a baseball bat over a base.'}, {'image_id': 294912, 'id': 504608, 'caption': 'a young baseball player swinging a bat at a ball'}, {'image_id': 294912, 'id': 505121, 'caption': 'A baseball player taking a swing at a ball'}]


In [ ]:


with open('/content/anns', 'w') as file:
    file.write(json.dumps(ann_caps, indent=4))

